In [1]:
# !pip install sqlalchemy==1.4.46 # с версией 2.0 не работает с pandas

In [43]:
import os

import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy

import datetime

In [105]:
posgres_username = "user"
posgres_password = "password"
hostname = "79.137.206.67:5432"
database_name = "traveldata"

engine_setup = 'postgresql+psycopg2://{}:{}@{}/{}'.format(
    posgres_username,
    posgres_password,
    hostname,
    database_name
)

In [23]:
# create engine 
engine = create_engine(engine_setup)

In [24]:
data_root = "./data/"

### download_data

In [49]:
data = pd.read_json(
    os.path.join(
        data_root,
        "TA_places_with_YANDEX_addresses.json"
    ), 
    orient='split',
    encoding="utf-8"
)

In [50]:
data.head()

,place_name,place_tags,place_address,place_rating,reviews_qty,reviews,place_url,longitude,latitude,yandex_place_address,postal_code,house,street,locality,province,country
0,Набережная Pеки Тура,[Пирсы и пляжные променады],"ул. Республики, 4, Тюмень 625003 Россия",4.5,639,"[{'review_title': 'Интересно прогуляться', 're...",https://www.tripadvisor.ru/Attraction_Review-g...,65.522371,57.161884,"Россия, Тюмень, улица Республики, 4",NaN,4,улица Республики,Тюмень,Тюменская область,Россия
1,Мост Влюбленных (Пешеходный),[Мосты],"ул.Республики и ул.Ленина, Тюмень 625026 Россия",4.5,516,"[{'review_title': 'Мост влюбленных. Красота', ...",https://www.tripadvisor.ru/Attraction_Review-g...,65.533429,57.154605,"Россия, Тюмень, улица Ленина",NaN,None,улица Ленина,Тюмень,Тюменская область,Россия
2,Сквер сибирских кошек,[Парки],"ул.Первомайская, 11, Тюмень 625000 Россия",4.0,298,"[{'review_title': 'Забавно', 'review_body': 'К...",https://www.tripadvisor.ru/Attraction_Review-g...,65.536708,57.153741,"Россия, Тюмень, Первомайская улица, 11",625000.0,11,Первомайская улица,Тюмень,Тюменская область,Россия
3,Свято-троицкий мужской монастырь,[Церкви и соборы],"Коммунистическая, 10, Тюмень 625001 Россия",5.0,58,"[{'review_title': 'БЛАГОДАТНОЕ МЕСТО', 'review...",https://www.tripadvisor.ru/Attraction_Review-g...,65.512840,57.169863,"Россия, Тюмень, Коммунистическая улица, 10",625001.0,10,Коммунистическая улица,Тюмень,Тюменская область,Россия
4,Памятник бездомной собаке,"[Культурные объекты и достопримечательности, П...","ул. Орджоникидзе, 63, Тюмень 625002 Россия",4.5,93,"[{'review_title': 'Памятник-копилка', 'review_...",https://www.tripadvisor.ru/Attraction_Review-g...,65.541999,57.149357,"Россия, Тюмень, улица Орджоникидзе, 63",625000.0,63,улица Орджоникидзе,Тюмень,Тюменская область,Россия


-----

In [51]:
places = data[["place_name"]]

In [52]:
places.to_sql(
    name="places", 
    con=engine, 
    schema="dbo",
    index=False,
    method="multi",
    if_exists="append" # append - добавляет строки в существующую таблицу /
)                      # replace - дропает таблицу и записывает заново

199

----

In [53]:
tags = data[["place_tags"]]

In [54]:
tags.insert(0, "place_id", data.index+1)

In [55]:
tags.columns = ["place_id", "tag"]

In [56]:
tags = tags.explode("tag").copy()

In [57]:
tags.head(10)

,place_id,tag
0,1,Пирсы и пляжные променады
1,2,Мосты
2,3,Парки
3,4,Церкви и соборы
4,5,Культурные объекты и достопримечательности
4,5,Памятники и статуи
5,6,Церкви и соборы
6,7,Театры
7,8,Парки
8,9,Культурные объекты и достопримечательности


In [58]:
tags.to_sql(
    name="tags", 
    con=engine, 
    schema="dbo",
    index=False,
    method="multi",
    if_exists="append" # append - добавляет строки в существующую таблицу /
)                      # replace - дропает таблицу и записывает заново

227

----

In [59]:
comments = data[["reviews"]]

In [60]:
comments.insert(0, "place_id", data.index+1)

In [61]:
comments = comments.explode("reviews").copy()

In [62]:
comments.columns = ["place_id", "comment"]

In [63]:
comments.to_sql(
    name="comments", 
    con=engine, 
    schema="dbo",
    index=False,
    method="multi",
    dtype={"comment": sqlalchemy.types.JSON},
    if_exists="append" # append - добавляет строки в существующую таблицу /
)                      # replace - дропает таблицу и записывает заново

3841

----

In [64]:
place_info = data[["place_rating",	"reviews_qty"]]

In [65]:
place_info.insert(0, "place_id", data.index+1)

In [66]:
place_info.insert(1, "info_datetime", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

In [72]:
place_info.columns = ["place_id", "info_datetime", "rating", "qnt_comments"]

In [73]:
place_info.head()

,place_id,info_datetime,rating,qnt_comments
0,1,2023-03-08 17:22:31,4.5,639
1,2,2023-03-08 17:22:31,4.5,516
2,3,2023-03-08 17:22:31,4.0,298
3,4,2023-03-08 17:22:31,5.0,58
4,5,2023-03-08 17:22:31,4.5,93


In [74]:
place_info.to_sql(
    name="place_info", 
    con=engine, 
    schema="dbo",
    index=False,
    method="multi",
    if_exists="append" # append - добавляет строки в существующую таблицу /
)                      # replace - дропает таблицу и записывает заново

199

----

In [76]:
coordinates = data[["longitude", "latitude"]].copy()

In [77]:
coordinates.insert(0, "place_id", data.index+1)

In [78]:
coordinates.to_sql(
    name="coordinates", 
    con=engine, 
    schema="dbo",
    index=False,
    method="multi",
    if_exists="append" # append - добавляет строки в существующую таблицу /
)                      # replace - дропает таблицу и записывает заново

199

----

In [79]:
addresses = data[["postal_code", "house", "street", "locality", "province", "country"]]

In [94]:
addresses.postal_code = addresses.postal_code.astype('Int64')

/tmp/ipykernel_8781/2864112592.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses.postal_code = addresses.postal_code.astype('Int64')


In [95]:
addresses

,place_id,postal_code,house,street,locality,province,country
0,1,<NA>,4,улица Республики,Тюмень,Тюменская область,Россия
1,2,<NA>,None,улица Ленина,Тюмень,Тюменская область,Россия
2,3,625000,11,Первомайская улица,Тюмень,Тюменская область,Россия
3,4,625001,10,Коммунистическая улица,Тюмень,Тюменская область,Россия
4,5,625000,63,улица Орджоникидзе,Тюмень,Тюменская область,Россия
...,...,...,...,...,...,...,...
194,195,<NA>,None,4-й км Салаирского тракта,Тюмень,Тюменская область,Россия
195,196,625007,137,улица Мельникайте,Тюмень,Тюменская область,Россия
196,197,625026,69Б,улица 50 лет ВЛКСМ,Тюмень,Тюменская область,Россия
197,198,625000,39,улица Республики,Тюмень,Тюменская область,Россия


In [97]:
addresses.insert(0, "place_id", data.index+1)

In [103]:
addresses.house.str.len().max()

6.0

In [104]:
addresses.to_sql(
    name="adresses", 
    con=engine, 
    schema="dbo",
    index=False,
    method="multi",
    if_exists="append" # append - добавляет строки в существующую таблицу /
)                      # replace - дропает таблицу и записывает заново

199